In [1]:
using CSV
using DataFrames

In [2]:
file_path = "dataset/simplified_Android_Malware.csv"

data = CSV.File(file_path, header=true) |> DataFrame

columns_to_drop = ["Flow ID", " Source IP", " Destination IP", " Timestamp"]

select!(data, Not(columns_to_drop))
dropmissing!(data)

Row,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
,Int64,Float64,Float64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String31
1,58310,443.0,6.0,96170080,8,12,3734.0,1499.0,1368.0,0.0,466.75,598.266,562.0,0.0,124.917,192.683,54.414,0.207965,5.06158e6,2.19676e7,9.57763e7,13.0,9.61701e7,1.37386e7,3.61753e7,9.57763e7,151.0,9.61347e7,8.73952e6,2.88792e7,9.58134e7,13.0,0.0,0.0,0,0,256,384,0.083186,0.124779,0.0,1368.0,314.333,481.687,2.32023e5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,330.05,466.75,124.917,256.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,3734.0,12.0,1499.0,1859.0,575.0,5.0,32.0,198506.0,0.0,198506.0,198506.0,9.57763e7,0.0,9.57763e7,9.57763e7,Android_Adware
2,11350,53.0,17.0,109573,1,1,34.0,199.0,34.0,34.0,34.0,0.0,199.0,199.0,199.0,0.0,2126.44,18.2527,109573.0,0.0,109573.0,109573.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,32,32,9.12634,9.12634,34.0,199.0,89.0,95.2628,9075.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,133.5,34.0,199.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,34.0,1.0,199.0,-1.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Android_Adware
3,26802,53.0,17.0,47706,1,1,37.0,225.0,37.0,37.0,37.0,0.0,225.0,225.0,225.0,0.0,5491.97,41.9234,47706.0,0.0,47706.0,47706.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,32,32,20.9617,20.9617,37.0,225.0,99.6667,108.542,11781.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,149.5,37.0,225.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,1.0,225.0,-1.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Android_Scareware
4,60386,443.0,6.0,20808858,8,8,343.0,5244.0,217.0,0.0,42.875,83.0309,1448.0,0.0,655.5,689.865,268.491,0.768903,1.38726e6,4.76123e6,1.85058e7,41.0,2.08089e7,2.97269e6,6.88919e6,1.85058e7,716.0,2.16747e6,309638.0,7.1848e5,1.93441e6,41.0,0.0,0.0,0,0,264,264,0.384452,0.384452,0.0,1448.0,328.647,558.774,3.12228e5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,349.188,42.875,655.5,264.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,343.0,8.0,5244.0,65535.0,31.0,2.0,32.0,2.30302e6,0.0,2.30302e6,2.30302e6,1.85058e7,0.0,1.85058e7,1.85058e7,Android_Adware
5,5177,53.0,17.0,30578,1,1,37.0,53.0,37.0,37.0,37.0,0.0,53.0,53.0,53.0,0.0,2943.29,65.4065,30578.0,0.0,30578.0,30578.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,32,

In [3]:
#show(data, allrows=false)
describe(data)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,Source Port,39136.0,0,43262.0,65517,0,Int64
2,Destination Port,5277.97,0.0,443.0,65165.0,0,Float64
3,Protocol,8.33805,0.0,6.0,17.0,0,Float64
4,Flow Duration,1.07323e7,2,549353.0,119977227,0,Int64
5,Total Fwd Packets,7.11518,1,2.0,3503,0,Int64
6,Total Backward Packets,10.6211,0,1.0,15439,0,Int64
7,Total Length of Fwd Packets,632.24,0.0,31.0,748480.0,0,Float64
8,Total Length of Bwd Packets,11585.8,0.0,17.0,2.23272e7,0,Float64
9,Fwd Packet Length Max,210.841,0.0,31.0,1460.0,0,Float64


### First approach: binary classification

In [4]:
function transform_binary_class(output_data)
    binary_labels = output_data .== "Benign"
    return binary_labels
end

input_data = Matrix(data[!, 1:80]);

output_data = data[!, :Label];

binary_labels = transform_binary_class(output_data)
@assert binary_labels isa BitVector
@assert input_data isa Matrix

In [5]:
include("functions.jl")
using Random

Random.seed!(42)

train_indexes, test_indexes = holdOut(size(input_data, 1), 0.2)

train_input = input_data[train_indexes, :]
train_output = binary_labels[train_indexes, :]

test_input = input_data[test_indexes, :]
test_output = binary_labels[test_indexes, :]

@assert size(test_input, 1) == size(test_output, 1)
@assert size(train_input, 1) == size(train_output, 1)

In [6]:
using ScikitLearn

@sk_import svm:SVC;
@sk_import tree:DecisionTreeClassifier;
@sk_import ensemble:VotingClassifier
@sk_import neighbors: KNeighborsClassifier;


[ Info: mkl not found, proceeding to installing non-mkl versions of sci-kit learn via Conda
[ Info: Running `conda install -y -c conda-forge 'scikit-learn>=1.2,<1.3'` in root environment


Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base conda




In [7]:
# models = Dict( "SVM" => SVC(probability=true), 
#          "DT"=> DecisionTreeClassifier(max_depth=4))
# base_models =  [ name for name in keys(models)]

# for key in keys(models)
#     model = models[key]
#     fit!(model, train_input, train_output)
#     acc = score(model, test_input, test_output)
#     println("$key: $(round(acc*100, digits=2))%")
# end

In [8]:
dt = DecisionTreeClassifier(max_depth=13)
fit!(dt, train_input, train_output)
score(dt, test_input, test_output)

0.9254885421060031

In [9]:
knn = KNeighborsClassifier(13)
fit!(knn, train_input, train_output)
score(knn, test_input, test_output)

/Users/alexdopico/.julia/conda/3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.9320961619569802

In [10]:
svm = SVC(kernel = "rbf")
fit!(svm, train_input, train_output)
score(svm, test_input, test_output)

/Users/alexdopico/.julia/conda/3/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9323773372697877

In [11]:
kFolds = 5
crossValidationIndexes = crossvalidation(vec(train_output), kFolds)

28450-element Vector{Int64}:
 4
 2
 5
 2
 5
 5
 2
 3
 5
 5
 2
 4
 4
 ⋮
 3
 3
 2
 4
 4
 3
 2
 5
 4
 2
 4
 4

In [13]:
include("functions.jl")
#svmParameters = Dict("modelType" => :SVM, "degree" => 3, "gamma" => 2, "kernel" => "linear", "C" => 0.6)
#dtParameters = Dict("modelType" => :DecisionTree, "maxDepth" => 2)
knnParameters = Dict("modelType" => :kNN, "numNeighboors" => 7)

# depths = [5, 7, 13, 15, 30, 64, 96, 128]
depths = [7, 13, 15]
for depth in depths
    knnParameters["numNeighboors"] = depth
    testAccuracy, testAccuracyStd = modelCrossValidation(knnParameters["modelType"], knnParameters, train_input, vec(train_output), crossValidationIndexes)
end

0.005291005291005291
0.0026455026455026454
0.005291005291005291
0.007936507936507936
0.005291005291005291
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [41]:
include("functions.jl")

#topologies = [[4], [8], [2,4], [12,12]]
topologies = [[80], [40, 20]]
annParameters = Dict("modelType" => :ANN, "maxEpochs" => 1000,
    "learningRate" => 0.01, "maxEpochsVal" => 50,
    "repetitions" => 50, "validationRatio" => 0.0,
    "transferFunctions" => fill(σ, 2))

for topology in topologies
    annParameters["topology"] = topology
    testAccuracy, testAccuracyStd, testRecall, testRecallStd = modelCrossValidation(annParameters["modelType"], annParameters, train_input, vec(train_output), crossValidationIndexes)
    println("$topology => $testAccuracy, $testAccuracyStd, $testRecall, $testRecallStd")
end


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [16]:
svmParameters = Dict("modelType" => :SVM, "degree" => 3, "gamma" => 2, "kernel" => "linear", "C" => 0.6)
testAccuracy, testAccuracyStd = modelCrossValidation(svmParameters["modelType"], svmParameters, train_input, vec(train_output), crossValidationIndexes)


In [9]:
include("functions.jl")

dtParameters = Dict("maxDepth" => 8)
knnParameters = Dict("numNeighboors" => 7)
svmParameters = Dict("kernel" => "linear", "C" => 0.5, "degree" => 3, "gamma" => 2)
Random.seed!(42)

kFolds = 2
crossValidationIndexes = crossvalidation(vec(train_output), kFolds)

trainClassEnsemble([:DecisionTree, :kNN, :SVM], [dtParameters, knnParameters, svmParameters], (train_input, train_output), crossValidationIndexes)


In [60]:
function crossvalidation_without_onehot(targets::AbstractArray{<:Any,1}, k::Int64)
    # Get the unique classes in the targets vector
    classes = unique(targets)
    # Initialize the index vector to zeros
    idx = Int.(zeros(length(targets)))
    
    # Iterate over the classes and perform stratified k-fold cross-validation
    for class in classes
        # Find the indices of the patterns that belong to the current class
        classIdx = (targets .== class)
        # Count the number of instances in the current class
        numInstances = sum(classIdx)
        # Check that there are enough instances per class to perform k-fold cross-validation
        @assert (numInstances .>= k) "There are not enough instances per class to perform a $(k)-fold cross-validation"
        # Perform stratified k-fold cross-validation on the patterns that belong to the current class
        classFolds = crossvalidation(numInstances, k)
        # Update the index vector with the indices of the folds for the current class
        idx[classIdx] .= classFolds
    end

    return idx
end

# Generate some test data
N = 20
targets = rand(["c1", "c2"], N)


# Call the crossvalidation function
crossvalidation_without_onehot(targets, 3)


20-element Vector{Int64}:
 1
 2
 2
 3
 1
 1
 3
 1
 1
 3
 2
 1
 2
 3
 3
 2
 1
 3
 2
 2

In [65]:
include("functions.jl")

crossValidationIndexes = crossvalidation_without_onehot(vec(train_output), 5)
minimum(crossValidationIndexes)

1

In [47]:
# for class in 1:size(train_output, 2)
oneHotEncoding(vec(train_output))

28450×1 BitMatrix:
 1
 0
 0
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 1
 0
 0

In [50]:
train_output isa AbstractArray{<:Any,1}

false